<a href="https://colab.research.google.com/github/MusfiqDehan/Bangla_PoS_Tagging/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- <h1 align="center"><strong>Bangla PoS Tagging</strong></h1> -->

# **Bangla PoS Tagging**

## Imports, Installations and Downloads

In [1]:
# Connect to Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Current working directory

%cd /content/drive/MyDrive/Colab Notebooks/CSE499/

/content/drive/MyDrive/Colab Notebooks/CSE499


In [4]:
# Installing Dependencies

!pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.3 MB 18.1 MB/s 
     |████████████████████████████████| 65 kB 4.3 MB/s 
     |████████████████████████████████| 53 kB 2.6 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=7b17bc4d9a782343490bbfa64a02de08767ef7cdb5e679dc6214b53ebd9bd73b
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [5]:
# Import Nesecessary Libraries

import nltk
from nltk.corpus import indian
from nltk.tokenize import word_tokenize
from nltk.tag import tnt
from googletrans import Translator
from pprint import pprint
import string

import pandas as pd
import numpy as np
import csv

In [7]:
# Downloading NLTK Corpus

nltk.download('indian')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Unzipping corpora/indian.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Preparing Dataset

In [8]:
# Show words from each langauge text

for text in indian.fileids():
    print(text, indian.words(text)[:30])
    print("\n")

bangla.pos ['মহিষের', 'সন্তান', ':', 'তোড়া', 'উপজাতি', '৷', 'বাসস্থান-ঘরগৃহস্থালি', 'তোড়া', 'ভাষায়', 'গ্রামকেও', 'বলে', '`', 'মোদ', "'", '৷', 'মোদের', 'আয়তন', 'খুব', 'বড়ো', 'নয়', '৷', 'প্রতি', 'মোদে', 'আছে', 'কিছু', 'কুঁড়েঘর', ',', 'সাধারণ', 'মহিষশালা', '৷']


hindi.pos ['पूर्ण', 'प्रतिबंध', 'हटाओ', ':', 'इराक', 'संयुक्त', 'राष्ट्र', '।', 'इराक', 'के', 'विदेश', 'मंत्री', 'ने', 'अमरीका', 'के', 'उस', 'प्रस्ताव', 'का', 'मजाक', 'उड़ाया', 'है', ',', 'जिसमें', 'अमरीका', 'ने', 'संयुक्त', 'राष्ट्र', 'के', 'प्रतिबंधों', 'को']


marathi.pos ["''", 'सनातनवाद्यांनी', 'व', 'प्रतिगाम्यांनी', 'समाज', 'रसातळाला', 'नेला', 'असताना', 'या', 'अंधारात', 'बाळशास्त्री', 'जांभेकर', 'यांनी', "'दर्पण'च्या", 'माध्यमातून', 'पहिली', 'ज्ञानज्योत', 'तेववली', ',', "''", 'असे', 'प्रतिपादन', 'नटसम्राट', 'प्रभाकर', 'पणशीकर', 'यांनी', 'केले', '.', 'दर्पणकार', 'बाळशास्त्री']


telugu.pos ['4', '.', 'ఆడిట్', 'నిర్వహణ', 'ఆడిటర్', 'ఒక', 'కొత్త', 'ఆడిట్', 'చేపట్టే', 'ముందు', 'సక్రమ', 'పద్ధతి', 'లో', 'కార్య', 'ప్రణాళికను', 'రూపొ

In [10]:
# Separating Bangla.Pos from Indian Corpus

tagged_set = 'bangla.pos'
word_set = indian.sents(tagged_set)
print(f"Total list in bangla.pos = {len(word_set)}")
print("\n")
pprint(word_set)

Total list in bangla.pos = 896


[['মহিষের', 'সন্তান', ':', 'তোড়া', 'উপজাতি', '৷'], ['বাসস্থান-ঘরগৃহস্থালি', 'তোড়া', 'ভাষায়', 'গ্রামকেও', 'বলে', '`', 'মোদ', "'", '৷'], ...]


In [ ]:
# Creating sentences and storing them in a list

bn_sentences = []

for sen in word_set:
    sen = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in sen]).strip()
    bn_sentences.append(sen)
    
print ('Total sentences in the tagged file are = ', len(bn_sentences))
print("\n")
pprint(bn_sentences)

Total sentences in the tagged file are =  896


['মহিষের সন্তান: তোড়া উপজাতি ৷',
 "বাসস্থান-ঘরগৃহস্থালি তোড়া ভাষায় গ্রামকেও বলে` মোদ' ৷",
 'মোদের আয়তন খুব বড়ো নয় ৷',
 'প্রতি মোদে আছে কিছু কুঁড়েঘর, সাধারণ মহিষশালা ৷',
 'আর গ্রামের বাইরে থাকে ডেয়ারি-মন্দির ৷',
 'আয়তনের তারতম্য অনুসারে গ্রামগুলি দু রকমের: এতূডমোদ( বড়ো গ্রাম) ওকিনমোদ( ছোট '
 'গ্রাম) ৷',
 'কোন কোন গ্রামের আবার ধর্মীয় বা_C\ufeffমহিষের সন্তান: তোড়া উপজাতি ৷',
 "িকে বলে` সোতি-মোদ' ৷",
 'কুঁড়েঘরগুলির আকার বাংলার বা ভারতের অন্য অঞ্চলের প্রচলিত কুঁড়ে ঘর নয় ৷',
 'এগুলি দেখতে শোয়ানো পিপের মতো ৷',
 'এক দিকের বাঁশের কাঠামো খিলানের মতো বেঁকে গিয়ে অন্যদিকের মাটিতে মিশেছে ৷',
 'সেগুলি বেত জাতীয় লতা দিয়ে পরস্পর বাঁধা ৷',
 'ঘরগুলি লম্বায় 18 ফুট, চওড়ায় 9 ফুট আর উচ্চতায় 10 ফুটের মতো ৷',
 'উপরের ঢালু চাল ঘাস-পাতা দিয়ে মোটা করে ছাওয়া ৷',
 'যাতে বৃষ্টির জল ভেতরে ঢুকতে না পারে ৷',
 'কুঁড়ের সামনের দিকে একটি মাত্র দরজা, আর কোথাও কোন রকমের জানলা বা ঘুলঘুলি নেই '
 '৷',
 'ঘরের ভেতরে ডানদিকের মেঝে কিছুটা উঁচু করে তোলা- সেটি তাদের শোবার জায়গা ৷',
 'বা

## Translating Sentences

In [ ]:
# Testing a random sentence for translation

translator = Translator()
test_result = translator.translate(str(bn_sentences[8]), src='bn', dest='en')
print(test_result)
print("\n")
print(f"{test_result.origin} -> {test_result.text}")

Translated(src=bn, dest=en, text=The size of the huts is not the current hut of Bengal or other regions of India., pronunciation=None, extra_data="{'confiden...")


কুঁড়েঘরগুলির আকার বাংলার বা ভারতের অন্য অঞ্চলের প্রচলিত কুঁড়ে ঘর নয় ৷ -> The size of the huts is not the current hut of Bengal or other regions of India.


In [ ]:
# Translating all the sentences and storing in a list

trans_to_eng = []
count = 0

for each_sen in bn_sentences:
    count = count + 1
    res = translator.translate(str(each_sen), src='bn', dest='en')
    trans_to_eng.append(res.text)
    print(f'{count} - {res.origin} -> {res.text}')

1 - মহিষের সন্তান: তোড়া উপজাতি ৷ -> Son of buffalo: bouquet tribes.
2 - বাসস্থান-ঘরগৃহস্থালি তোড়া ভাষায় গ্রামকেও বলে` মোদ' ৷ -> The housekeeper in the house of housing and the village is called 'Wine'.
3 - মোদের আয়তন খুব বড়ো নয় ৷ -> The size of the Moder is not very big.
4 - প্রতি মোদে আছে কিছু কুঁড়েঘর, সাধারণ মহিষশালা ৷ -> There are some horses in Mod, the general builder.
5 - আর গ্রামের বাইরে থাকে ডেয়ারি-মন্দির ৷ -> And the Dairy-Temple is out of the village.
6 - আয়তনের তারতম্য অনুসারে গ্রামগুলি দু রকমের: এতূডমোদ( বড়ো গ্রাম) ওকিনমোদ( ছোট গ্রাম) ৷ -> Villages according to the varies of rural areas: Upholstered (big village) Okinomod (small village).
7 - কোন কোন গ্রামের আবার ধর্মীয় বা_C﻿মহিষের সন্তান: তোড়া উপজাতি ৷ -> Some villages are again the children of religious or C buffalo: bouquet tribes.
8 - িকে বলে` সোতি-মোদ' ৷ -> Who says 'Si-Mod'.
9 - কুঁড়েঘরগুলির আকার বাংলার বা ভারতের অন্য অঞ্চলের প্রচলিত কুঁড়ে ঘর নয় ৷ -> The size of the huts is not the current hut of Bengal or other regio

In [ ]:
# Checking the length of original and translated list

print(f"Length of original Bangla list = {len(bn_sentences)}")
print(f"Length of translated to English list = {len(trans_to_eng)}")

Length of original Bangla list = 896
Length of translated to English list = 896


## Converting into DataFrame

In [ ]:
# Converting bn_sentences list into a csv file

# df1 = pd.DataFrame(bn_sentences, columns=["bn_sentences"])
# df1.to_csv('bn_sentences.csv', index=False)

In [ ]:
df1 = pd.read_csv('csv_files/bn_sentences.csv')
df1.head()

,bn_sentences
0,মহিষের সন্তান: তোড়া উপজাতি ৷
1,বাসস্থান-ঘরগৃহস্থালি তোড়া ভাষায় গ্রামকেও বলে` ...
2,মোদের আয়তন খুব বড়ো নয় ৷
3,"প্রতি মোদে আছে কিছু কুঁড়েঘর, সাধারণ মহিষশালা ৷"
4,আর গ্রামের বাইরে থাকে ডেয়ারি-মন্দির ৷


In [ ]:
df1_dict = df1.to_dict()
df1_dict_5_list = []
for i in range(5):
    dct_out = df1_dict['bn_sentences'][i]
    print(dct_out, type(dct_out))
    df1_dict_5_list.append(dct_out)

pprint(df1_dict_5_list)

মহিষের সন্তান: তোড়া উপজাতি ৷ <class 'str'>
বাসস্থান-ঘরগৃহস্থালি তোড়া ভাষায় গ্রামকেও বলে` মোদ' ৷ <class 'str'>
মোদের আয়তন খুব বড়ো নয় ৷ <class 'str'>
প্রতি মোদে আছে কিছু কুঁড়েঘর, সাধারণ মহিষশালা ৷ <class 'str'>
আর গ্রামের বাইরে থাকে ডেয়ারি-মন্দির ৷ <class 'str'>
['মহিষের সন্তান: তোড়া উপজাতি ৷',
 "বাসস্থান-ঘরগৃহস্থালি তোড়া ভাষায় গ্রামকেও বলে` মোদ' ৷",
 'মোদের আয়তন খুব বড়ো নয় ৷',
 'প্রতি মোদে আছে কিছু কুঁড়েঘর, সাধারণ মহিষশালা ৷',
 'আর গ্রামের বাইরে থাকে ডেয়ারি-মন্দির ৷']


In [ ]:
# Converting trans_to_eng list into a csv file

# df2 = pd.DataFrame(trans_to_eng, columns=["trans_to_eng"])
# df2.to_csv('trans_to_eng.csv', index=False)

In [ ]:
df2 = pd.read_csv('csv_files/trans_to_eng.csv')
df2.head()

,trans_to_eng
0,Son of buffalo: bouquet tribes.
1,The housekeeper in the house of housing and th...
2,The size of the Moder is not very big.
3,"There are some horses in Mod, the general buil..."
4,And the Dairy-Temple is out of the village.


In [ ]:
df2_dict = df2.to_dict()
df2_dict_5_list = []
for i in range(5):
    dct_out = df2_dict['trans_to_eng'][i]
    print(dct_out, type(dct_out))
    df2_dict_5_list.append(dct_out)

pprint(df2_dict_5_list)

Son of buffalo: bouquet tribes. <class 'str'>
The housekeeper in the house of housing and the village is called 'Wine'. <class 'str'>
The size of the Moder is not very big. <class 'str'>
There are some horses in Mod, the general builder. <class 'str'>
And the Dairy-Temple is out of the village. <class 'str'>
['Son of buffalo: bouquet tribes.',
 "The housekeeper in the house of housing and the village is called 'Wine'.",
 'The size of the Moder is not very big.',
 'There are some horses in Mod, the general builder.',
 'And the Dairy-Temple is out of the village.']


In [ ]:
# Merged original and translated files

# df3 = pd.concat([df1, df2], axis=1)
# df3.to_csv("bn_eng.csv", index=False)

In [ ]:
df3 = pd.read_csv('csv_files/bn_eng.csv')
df3.head()

,bn_sentences,trans_to_eng
0,মহিষের সন্তান: তোড়া উপজাতি ৷,Son of buffalo: bouquet tribes.
1,বাসস্থান-ঘরগৃহস্থালি তোড়া ভাষায় গ্রামকেও বলে` ...,The housekeeper in the house of housing and th...
2,মোদের আয়তন খুব বড়ো নয় ৷,The size of the Moder is not very big.
3,"প্রতি মোদে আছে কিছু কুঁড়েঘর, সাধারণ মহিষশালা ৷","There are some horses in Mod, the general buil..."
4,আর গ্রামের বাইরে থাকে ডেয়ারি-মন্দির ৷,And the Dairy-Temple is out of the village.


## PoS Tagging of English Sentences

In [ ]:
df2_row, df2_col = df2.shape
eng_pos_tagged_sentences = []

count2 = 0
for i in range(df2_row):
    count2 = count2 + 1
    eng_sen = df2_dict['trans_to_eng'][i]
    eng_words = word_tokenize(str(eng_sen))
    pos_tagged_sen = nltk.pos_tag(eng_words)
    eng_pos_tagged_sentences.append(pos_tagged_sen)  
    print(f"{count2} - {pos_tagged_sen}")


# pprint(eng_pos_tagged_sentences)

1 - [('Son', 'NN'), ('of', 'IN'), ('buffalo', 'NN'), (':', ':'), ('bouquet', 'NN'), ('tribes', 'NNS'), ('.', '.')]
2 - [('The', 'DT'), ('housekeeper', 'NN'), ('in', 'IN'), ('the', 'DT'), ('house', 'NN'), ('of', 'IN'), ('housing', 'NN'), ('and', 'CC'), ('the', 'DT'), ('village', 'NN'), ('is', 'VBZ'), ('called', 'VBN'), ("'Wine", 'NN'), ("'", "''"), ('.', '.')]
3 - [('The', 'DT'), ('size', 'NN'), ('of', 'IN'), ('the', 'DT'), ('Moder', 'NNP'), ('is', 'VBZ'), ('not', 'RB'), ('very', 'RB'), ('big', 'JJ'), ('.', '.')]
4 - [('There', 'EX'), ('are', 'VBP'), ('some', 'DT'), ('horses', 'NNS'), ('in', 'IN'), ('Mod', 'NNP'), (',', ','), ('the', 'DT'), ('general', 'JJ'), ('builder', 'NN'), ('.', '.')]
5 - [('And', 'CC'), ('the', 'DT'), ('Dairy-Temple', 'NNP'), ('is', 'VBZ'), ('out', 'IN'), ('of', 'IN'), ('the', 'DT'), ('village', 'NN'), ('.', '.')]
6 - [('Villages', 'NNS'), ('according', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('varies', 'NNS'), ('of', 'IN'), ('rural', 'JJ'), ('areas', 'NNS'), (':', '

In [ ]:
# Lengths of English Sentences

maximum = max(map(len, eng_pos_tagged_sentences))
minimum = min(map(len, eng_pos_tagged_sentences))

print(f"Maximum Length = {maximum}")
print(f"Minimum Length = {minimum}")

Maximum Length = 74
Minimum Length = 1


In [11]:
# Lengths of Bangla Sentences

max_bn = max(map(len, word_set))
min_bn = min(map(len, word_set))

print(f"Maximum Length = {max_bn}")
print(f"Minimum Length = {min_bn}")

Maximum Length = 58
Minimum Length = 2


In [ ]:
line705 = [('When', 'WRB'), ('the', 'DT'), ('rice', 'NN'), ('is', 'VBZ'), ('twenty-five', 'JJ'), ('in', 'IN'), ('twenty-five', 'JJ'), (',', ','), ('people', 'NNS'), ('did', 'VBD'), ('not', 'RB'), ('get', 'VB'), ('their', 'PRP$'), ('hands', 'NNS'), ('on', 'IN'), ('the', 'DT'), ('head', 'NN'), (',', ','), ('Capas', 'NNP'), ('cotton', 'NN'), ('did', 'VBD'), ('not', 'RB'), ('get', 'VB'), ('fourteen', 'JJ'), ('pao', 'NN'), (',', ','), ('but', 'CC'), ('a', 'DT'), ('pair', 'NN'), ('of', 'IN'), ('good', 'JJ'), ('white', 'JJ'), ('dhoti', 'NN'), ('prices', 'NNS'), ('would', 'MD'), ('have', 'VB'), ('been', 'VBN'), ('worth', 'JJ'), ('eleven', 'RB'), (',', ','), ('then', 'RB'), ('the', 'DT'), ('price', 'NN'), ('of', 'IN'), ('a', 'DT'), ('pair', 'NN'), ('was', 'VBD'), ('very', 'RB'), ('low', 'JJ'), (',', ','), ('so', 'IN'), ('the', 'DT'), ('price', 'NN'), ('was', 'VBD'), ('very', 'RB'), ('low', 'JJ'), (',', ','), ('so', 'RB'), ('do', 'VB'), ('not', 'RB'), ('despise', 'VB'), ('it', 'PRP'), ('againLet', 'PRP'), ("'s", 'VBZ'), ('see', 'VB'), ('what', 'WP'), ('is', 'VBZ'), ('hiding', 'VBG'), ('in', 'IN'), ('the', 'DT'), ('last', 'JJ'), ('Hijabi', 'NNP'), ('design', 'NN'), ('.', '.')]
len(line705)

74

In [ ]:
c74 = []

for i in range(74):
    c = f"c{i}"
    c74.append(c)

print(c74)


['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73']


In [ ]:
# Converting trans_to_eng list into a csv file

# df4 = pd.DataFrame(eng_pos_tagged_sentences, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73'])
# df4.to_csv('eng_pos_tagged_sentences.csv', index=False)

In [ ]:
df4 = pd.read_csv('csv_files/eng_pos_tagged_sentences.csv')
df4.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,c31,c32,c33,c34,c35,c36,c37,c38,c39,c40,c41,c42,c43,c44,c45,c46,c47,c48,c49,c50,c51,c52,c53,c54,c55,c56,c57,c58,c59,c60,c61,c62,c63,c64,c65,c66,c67,c68,c69,c70,c71,c72,c73
0,"(Son, NN)","(of, IN)","(buffalo, NN)","(:, :)","(bouquet, NN)","(tribes, NNS)","(., .)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"(The, DT)","(housekeeper, NN)","(in, IN)","(the, DT)","(house, NN)","(of, IN)","(housing, NN)","(and, CC)","(the, DT)","(village, NN)","(is, VBZ)","(called, VBN)","('Wine, NN)","(', '')","(., .)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"(The, DT)","(size, NN)","(of, IN)","(the, DT)","(Moder, NNP)","(is, VBZ)","(not, RB)","(very, RB)","(big, JJ)","(., .)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"(There, EX)","(are, VBP)","(some, DT)","(horses, NNS)","(in, IN)","(Mod, NNP)","(,, ,)","(the, DT)","(general, JJ)","(builder, NN)","(., .)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"(And, CC)","(the, DT)","(Dairy-Temple, NNP)","(is, VBZ)","(out, IN)","(of, IN)","(the, DT)","(village, NN)","(., .)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
df4.shape

In [ ]:
# Merged original and translated and pos files

# df5 = pd.concat([df3, df4], axis=1)
# df5.to_csv("bn_eng_pos.csv", index=False)

In [ ]:
df5 = pd.read_csv('csv_files/bn_eng_pos.csv')
df5.head()

,bn_sentences,trans_to_eng,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,c31,c32,c33,c34,c35,c36,c37,c38,c39,c40,c41,c42,c43,c44,c45,c46,c47,c48,c49,c50,c51,c52,c53,c54,c55,c56,c57,c58,c59,c60,c61,c62,c63,c64,c65,c66,c67,c68,c69,c70,c71,c72,c73
0,মহিষের সন্তান: তোড়া উপজাতি ৷,Son of buffalo: bouquet tribes.,"('Son', 'NN')","('of', 'IN')","('buffalo', 'NN')","(':', ':')","('bouquet', 'NN')","('tribes', 'NNS')","('.', '.')",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,বাসস্থান-ঘরগৃহস্থালি তোড়া ভাষায় গ্রামকেও বলে` ...,The housekeeper in the house of housing and th...,"('The', 'DT')","('housekeeper', 'NN')","('in', 'IN')","('the', 'DT')","('house', 'NN')","('of', 'IN')","('housing', 'NN')","('and', 'CC')","('the', 'DT')","('village', 'NN')","('is', 'VBZ')","('called', 'VBN')","(""'Wine"", 'NN')","(""'"", ""''"")","('.', '.')",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,মোদের আয়তন খুব বড়ো নয় ৷,The size of the Moder is not very big.,"('The', 'DT')","('size', 'NN')","('of', 'IN')","('the', 'DT')","('Moder', 'NNP')","('is', 'VBZ')","('not', 'RB')","('very', 'RB')","('big', 'JJ')","('.', '.')",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"প্রতি মোদে আছে কিছু কুঁড়েঘর, সাধারণ মহিষশালা ৷","There are some horses in Mod, the general buil...","('There', 'EX')","('are', 'VBP')","('some', 'DT')","('horses', 'NNS')","('in', 'IN')","('Mod', 'NNP')","(',', ',')","('the', 'DT')","('general', 'JJ')","('builder', 'NN')","('.', '.')",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,আর গ্রামের বাইরে থাকে ডেয়ারি-মন্দির ৷,And the Dairy-Temple is out of the village.,"('And', 'CC')","('the', 'DT')","('Dairy-Temple', 'NNP')","('is', 'VBZ')","('out', 'IN')","('of', 'IN')","('the', 'DT')","('village', 'NN')","('.', '.')",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df5.shape

(896, 76)